Imports

In [2]:
# Import necessari
import cv2
import numpy as np
import os
import pickle
import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from sklearn.model_selection import train_test_split
import json
import torchvision
from torchvision.models.detection import maskrcnn_resnet50_fpn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

Canny edge detection

In [20]:
def get_edge_image(image, low_threshold, high_threshold):
    # Convert the image to grayscale if it's not already
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Apply Canny edge detection
    edges = cv2.Canny(gray, low_threshold, high_threshold)
    
    return edges

# Load an image
image = cv2.imread(r'C:\Users\loren\Downloads\APKLOT-master\APKLOT-master\1. Satellite\Dataset\parcheggi\Dataset\402492112_training.png')

# Get the edge image
edge_image = get_edge_image(image, low_threshold=250, high_threshold=350)

# Display the edge image
cv2.imshow('Edge Image', edge_image)
cv2.waitKey(0)

-1

Store canny edge

In [ ]:
dataset_path = r'C:\Users\loren\Downloads\APKLOT-master\APKLOT-master\1. Satellite\Dataset\parcheggi\Dataset'
edge_path = r'C:\Users\loren\Downloads\APKLOT-master\APKLOT-master\1. Satellite\Dataset\parcheggi\Dataset_edge'

for filename in os.listdir(dataset_path):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        input_image_path = os.path.join(dataset_path, filename)
        image = cv2.imread(input_image_path)
        # Ottieni l'immagine del bordo
        edge_image = get_edge_image(image, low_threshold=250, high_threshold=350)
        # Salva l'immagine del bordo nella cartella di output
        output_image_path = os.path.join(edge_path, filename)
        cv2.imwrite(output_image_path, edge_image)

print("Il rilevamento dei bordi è stato applicato a tutte le immagini nella cartella.")

Dataset preparation

In [ ]:
# Custom dataset class
class ParkingLotDataset(Dataset):
    def __init__(self, image_paths, mask_paths, transforms=None):
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.transforms = transforms

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        mask_path = self.mask_paths[idx]

        # Load image
        image = cv2.imread(image_path)
        #image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Load masks
        with open(mask_path, 'r') as f:
            masks_data = json.load(f)

        masks = []
        for shape in masks_data['shapes']:
            points = np.array(shape['points'])
            mask = np.zeros_like(image[:, :, 0])
            cv2.fillPoly(mask, [points.astype(np.int32)], 1)
            masks.append(mask)

        masks = np.stack(masks, axis=0).astype(np.float32)

        # Apply transformations
        if self.transforms:
            image = self.transforms(image)
            masks = self.transforms(masks)

        return image, masks

# Initialize variables to store max height and width
max_height = 0
max_width = 0

# Iterate through all images in the directory
for filename in os.listdir('C:/Users/loren/Downloads/APKLOT-master/APKLOT-master/1. Satellite/Dataset/parcheggi/dataloader'):
    if filename.endswith('.png') or filename.endswith('.jpg'):
        # Load the image
        image_path = os.path.join('C:/Users/loren/Downloads/APKLOT-master/APKLOT-master/1. Satellite/Dataset/parcheggi/dataloader', filename)
        image = cv2.imread(image_path)

        # Get height and width of the image
        height, width, _ = image.shape

        # Update max_height and max_width if necessary
        if height > max_height:
            max_height = height
        if width > max_width:
            max_width = width

print("Maximum Height:", max_height)
print("Maximum Width:", max_width)

# Data transformations
transform = transforms.Compose([
    transforms.Resize((max_height, max_width)),  # Resize to maximum dimensions
    transforms.Pad(padding),  # Pad images to make them all the same size
    transforms.ToTensor()
])

# Combine image and mask paths
image_paths = []
mask_paths = []
for filename in os.listdir('C:/Users/loren/Downloads/APKLOT-master/APKLOT-master/1. Satellite/Dataset/parcheggi/dataloader'):
    if filename.endswith('.png'):
        image_path = 'C:/Users/loren/Downloads/APKLOT-master/APKLOT-master/1. Satellite/Dataset/parcheggi/dataloader'+ '/' + filename
        mask_path = 'C:/Users/loren/Downloads/APKLOT-master/APKLOT-master/1. Satellite/Dataset/parcheggi/dataloader'+ '/' + filename.replace('.png', '.json')
        image_paths.append(image_path)
        mask_paths.append(mask_path)





DATASET CLASS

In [3]:
# Custom dataset class
class ParkingLotDataset(Dataset):
    def __init__(self, image_paths, mask_paths, transforms=None):
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.transforms = transforms

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        mask_path = self.mask_paths[idx]

        # Load image
        image = cv2.imread(image_path)
        #image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Load masks
        with open(mask_path, 'r') as f:
            masks_data = json.load(f)

        masks = []
        for shape in masks_data['shapes']:
            points = np.array(shape['points'])
            mask = np.zeros_like(image[:, :, 0])
            cv2.fillPoly(mask, [points.astype(np.int32)], 1)
            masks.append(mask)

        masks = np.stack(masks, axis=0).astype(np.float32)

        # Apply transformations
        if self.transforms:
            image = self.transforms(image)
            masks = self.transforms(masks)

        return image, masks

DATASET SLPIT

In [19]:
#controllo il massimo valore di altezza e larghezza per fare il padding
import cv2
import os

# Directory containing the images
data_dir = 'dataloader'

# Initialize variables to store max height and width
max_height = 0
max_width = 0

# Iterate through all images in the directory
for filename in os.listdir(data_dir):
    if filename.endswith('.png') or filename.endswith('.jpg'):
        # Load the image
        image_path = os.path.join(data_dir, filename)
        image = cv2.imread(image_path)

        # Get height and width of the image
        height, width, _ = image.shape

        # Update max_height and max_width if necessary
        if height > max_height:
            max_height = height
        if width > max_width:
            max_width = width

print("Maximum Height:", max_height)
print("Maximum Width:", max_width)

Maximum Height: 2108
Maximum Width: 2113


In [15]:
# magari rifare il dataloader con due cartelle
image_path = 'dataloader'

# Combine image and mask paths
#NON VA BENE COSI
image_paths = []
mask_paths = []
for filename in os.listdir('dataloader'):
    if filename.endswith('.png'):
        image_path = 'dataloader'+ '/' + filename
        mask_path = 'dataloader'+ '/' + filename.replace('.png', '.json')
        image_paths.append(image_path)
        mask_paths.append(mask_path)
print(image_paths)
print(mask_paths)
# Data transformations

transform = transforms.Compose([
    #transforms.Resize((max_height, max_width)),  # Resize to maximum dimensions
    #transforms.Pad(padding),  # Pad images to make them all the same size
    transforms.ToTensor()
])
# Create dataset
dataset = ParkingLotDataset(image_paths, mask_paths, transforms=transform)

# Split image and mask paths into train and test sets
train_images, test_images, train_masks, test_masks = train_test_split(dataset.image_paths, dataset.mask_paths, test_size=0.2, random_state=42)

# Further split train set into train and validation sets
train_images, val_images, train_masks, val_masks = train_test_split(train_images, train_masks, test_size=0.2, random_state=42)

# Define datasets and data loaders for train, validation, and test sets
train_dataset = ParkingLotDataset(train_images, train_masks, transforms=transform)
val_dataset = ParkingLotDataset(val_images, val_masks, transforms=transform)
test_dataset = ParkingLotDataset(test_images, test_masks, transforms=transform)

# Create data loaders
train_data_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_data_loader = DataLoader(val_dataset, batch_size=4, shuffle=True)
test_data_loader = DataLoader(test_dataset, batch_size=4, shuffle=True)

['dataloader/101881731.png', 'dataloader/103109948.png', 'dataloader/103614205.png', 'dataloader/104101927.png', 'dataloader/104357814.png', 'dataloader/104357815.png', 'dataloader/104357817.png', 'dataloader/104907575.png', 'dataloader/116016718.png', 'dataloader/119422952.png', 'dataloader/119917133.png', 'dataloader/126381586.png', 'dataloader/126381588.png', 'dataloader/126381594.png', 'dataloader/127649594.png', 'dataloader/127649606.png', 'dataloader/130730813.png', 'dataloader/132613254.png', 'dataloader/134714700.png', 'dataloader/136128142.png', 'dataloader/136128143.png', 'dataloader/136128144.png', 'dataloader/137007454.png', 'dataloader/137007455.png', 'dataloader/139146552.png', 'dataloader/139146568.png', 'dataloader/144384815.png', 'dataloader/144848258.png', 'dataloader/147397541.png', 'dataloader/148521975.png', 'dataloader/148788621.png', 'dataloader/151179182.png', 'dataloader/152582726.png', 'dataloader/153028544.png', 'dataloader/15309700.png', 'dataloader/15311494

In [14]:
#print delle immagini
import matplotlib.pyplot as plt

# Display images and masks from the train data loader
for images, masks in train_data_loader:
    for i in range(images.size(0)):
        image = images[i].permute(1, 2, 0).numpy()
        mask = masks[i].permute(1, 2, 0).numpy()

        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
        ax1.imshow(image)
        ax1.set_title('Image')
        ax2.imshow(mask.squeeze(), cmap='gray')
        ax2.set_title('Mask')
        plt.show()
        break  # Display only one batch
    break  # Display only one batch

FileNotFoundError: [Errno 2] No such file or directory: 'dataloader/203436248.json'

Model

In [21]:
import torch
import torch.nn as nn
import torchvision
from torchvision.models.detection import MaskRCNN_ResNet50_FPN_Weights
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# Load the pre-trained model weights
model_weights = MaskRCNN_ResNet50_FPN_Weights.DEFAULT
model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights=model_weights)

# Get the number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features

# Replace the pre-final layer with a new one for binary classification
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes=2)

# Set the model in training mode
model.train()

# Define the loss function (e.g., Dice Loss)
def dice_loss(inputs, targets, smooth=1e-5):
    inputs = inputs.view(-1)
    targets = targets.view(-1)
    intersection = (inputs * targets).sum()
    union = inputs.sum() + targets.sum()
    dice = (2 * intersection + smooth) / (union + smooth)
    return 1 - dice

# Define the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    epoch_loss = 0.0
    for images, masks in train_data_loader:
        images = list(image.to(device) for image in images)
        masks = [{k: v.to(device) for k, v in t.items()} for t in masks]

        optimizer.zero_grad()

        outputs = model(images, masks)
        loss = sum(loss for loss in outputs.values())

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss/len(train_loader)}')

FileNotFoundError: [Errno 2] No such file or directory: 'dataloader/397174539.json'

Train

In [17]:
# Define the number of epochs
num_epochs = 10

for epoch in range(num_epochs):
    # Training
    model.train()
    for images, targets in train_data_loader:
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        for images, targets in val_data_loader:
            images = list(image.to(device) for image in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())

            # You can add here code to compute metrics and print them


NameError: name 'model' is not defined